In [14]:
import os
HOME = os.getcwd()            #working directory
print(HOME)

e:\car-parking-slot-occupancy-detection-using-YOLOv8---openCV-main


In [15]:
import numpy as np
import cv2


In [16]:
camera = cv2.VideoCapture(1)

In [17]:
from ultralytics import YOLO

from IPython.display import display, Image                     #used for displaying images 

In [18]:
file = open("final_coordinates.txt")

print("[INFO] Loading parking coordinates ...")

lines = file.readlines()

lines = [line.strip() for line in lines]    #Strips leading and trailing whitespace from each line in the lines list.

total_parking_lots = len(lines)            #Calculates the total number of parking lots by counting the number of lines and assigns this value to the variable total_parking_lots.

parking_lot_coords = list()

for i in range(len(lines)):
    
    coords = lines[i].split()
    top_left = (int(coords[0]), int(coords[1]))  #Converts the first two elements of coords to integers and stores them as a tuple representing the top-left coordinate.
    top_right = (int(coords[2]), int(coords[3]))
    bottom_right = (int(coords[4]), int(coords[5]))
    bottom_left = (int(coords[6]), int(coords[7]))
    coord = np.array([top_left,top_right, bottom_right,bottom_left]) #Creates a NumPy array from the four corner coordinates and assigns it to the variable coord.
    parking_lot_coords.append(coord)
print(parking_lot_coords)

[INFO] Loading parking coordinates ...
[array([[186,  58],
       [185, 277],
       [ 44, 278],
       [ 48,  61]]), array([[190,  55],
       [352,  54],
       [350, 276],
       [187, 277]]), array([[353,  52],
       [466,  53],
       [462, 275],
       [354, 277]]), array([[188, 577],
       [ 53, 572],
       [ 41, 359],
       [183, 361]]), array([[350, 359],
       [352, 581],
       [190, 576],
       [184, 359]]), array([[465, 362],
       [462, 579],
       [350, 578],
       [349, 362]]), array([[757,  55],
       [870,  57],
       [866, 283],
       [757, 280]]), array([[ 872,   57],
       [1035,   63],
       [1035,  281],
       [ 868,  279]]), array([[1036,   62],
       [1174,   70],
       [1180,  285],
       [1039,  282]]), array([[855, 587],
       [740, 585],
       [750, 365],
       [864, 365]]), array([[1014,  585],
       [ 857,  585],
       [ 865,  365],
       [1026,  368]]), array([[1155,  584],
       [1013,  586],
       [1030,  367],
       [1170,  

In [19]:


# Load your models
#model1 = YOLO('hanifv8.pt')
model1 = YOLO('wrong_car.pt')
#model1 = YOLO('car.pt')
while True:
    occupied_slots_class_0 = 0
    occupied_slots_class_1 = 0
    available_slots = len(parking_lot_coords)

    ret, frame = camera.read()

    if not ret:
        print("[ERROR] Failed to initialize camera.")
        cv2.destroyAllWindows()
        break

    if ret:
        frame = cv2.resize(frame, (1300, 650))

        for i in range(len(parking_lot_coords)):
            cv2.polylines(frame, [np.array(parking_lot_coords[i], np.int32)], True, (0, 255, 0), 2)

        results_1 = model1.predict(frame)
        # results_2 = model2.predict(frame)
        result = results_1[0]

        for box in result.boxes:
            class_id = box.cls[0].item()
            if class_id in [0.0, 1.0]:
                cord = box.xyxy[0].tolist()
                cord = [round(x) for x in cord]
                cx = (cord[0] + cord[2]) // 2
                cy = (cord[1] + cord[3]) // 2
                point = (cx, cy)

                for arr in parking_lot_coords:
                    if cv2.pointPolygonTest(np.array(arr, np.int32), point, False) >= 0:
                        if class_id == 0.0:
                            occupied_slots_class_0 += 1
                        elif class_id == 1.0:
                            occupied_slots_class_1 += 1
                        available_slots -= 1
                        cv2.polylines(frame, [np.array(arr, np.int32)], True, (0, 0, 255), 2)

        total_occupied_slots = occupied_slots_class_0 + occupied_slots_class_1

        cv2.putText(frame, "Wrong Cars Parked: " + str(occupied_slots_class_0), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 2)
        cv2.putText(frame, "Right Cars Parked: " + str(occupied_slots_class_1), (50, 100), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 2)
        cv2.putText(frame, "Available Slots: " + str(available_slots), (50, 150), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0,255), 2)

        cv2.imshow("Camera", frame)

        key = cv2.waitKey(1)

        if key % 256 == 27:
            # ESC
            print("[INFO] Camera terminated.")
            cv2.destroyAllWindows()
            break



0: 320x640 3 cars, 203.4ms
Speed: 5.9ms preprocess, 203.4ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 4 cars, 317.8ms
Speed: 9.0ms preprocess, 317.8ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 4 cars, 182.3ms
Speed: 6.4ms preprocess, 182.3ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 4 cars, 181.0ms
Speed: 4.4ms preprocess, 181.0ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 4 cars, 183.2ms
Speed: 5.8ms preprocess, 183.2ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 4 cars, 181.7ms
Speed: 5.8ms preprocess, 181.7ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 4 cars, 173.2ms
Speed: 4.7ms preprocess, 173.2ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 4 cars, 175.4ms
Speed: 6.1ms preprocess, 175.4ms inference, 1.0ms postprocess per image at shape (1, 3, 32